In [76]:
pip install keybert

In [77]:
pip install -U sentence-transformers

In [78]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('punkt')
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import spacy
from nltk.stem.porter import PorterStemmer
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Extracting Data**

In [79]:
file=open('/content/use_case_online.txt','r')
file.seek(0)
use_case=file.read()

In [80]:
file=open('/content/func_req_online.txt','r')
file.seek(0)
func_req=file.read()

In [81]:
file = open('/content/suggestions.txt','r')
file.seek(0)
sugg=file.read()

In [82]:
use_case

'\ufeffOnline_Auction(Use_case_dataset)\n\n\n1) Login or create account : User can login into already registered account or create a new account to login. If password is correct and verified then user can login into account immediately. If password is wrong then it will show wrong password message and it will go back to login page.\n\n\n2) Get bid history : User can see and get his/her bidding history.\n3) Get list of auction : User can see item list which is to be auctioned. \n4) Place an item for bid : User, who is an owner, can place an item for bidding in auction for   other users-Date : When the user place the item for bidding, that date will be saved in database . Time : When the user place the item for bidding, that time will be saved in database. Picture of item : The user have to give pictures of the item that he/she want to place for bid. Withdraw: The user can withdraw the item that has gone for bidding from auction. Initial Price: The user can set the initial price for the 

In [83]:
func_req

'\ufeff1. SET UP ACCOUNT : When the user wants to enter into this software he/she should go through the account setup phrase. In this section the user will provide his/her personal details like home address, email, phone number, DOB and set up a password for the account,The password criteria should match as follows:  It should contain an uppercase alphabet, at least one special character, a numeric value.  It should be at least 8 character long.\n2. SIGN IN : Here the user can sign in to previously registered account by providing email address and password.  If login credentials are valid the user is directed to the app home page.  If there the credentials are invalid an error login message is displayed.  The user can reset his/her password by providing the registered email address. A password reset link is send to the user via email.  For sign in internet availability is required\n3. GET BID HISTORY : All user can get recent 100 bid history (picture of an item, initial price, after au

In [84]:
sugg

'3. GET BID HISTORY : All user can get recent 100 bid history (picture of an item, initial price, after auction price,date and time of auction,user name or anonymous). Old history will automatically get deleted when a new bid is added.\n4.login in with gmail.\n GET LIST OF AUCTION : Users get all the upcoming auction list of 100 product here with date and time.'

In [85]:
use_case_list=use_case.split("\n")
func_req_list=func_req.split("\n")
use_case_list

['\ufeffOnline_Auction(Use_case_dataset)',
 '',
 '',
 '1) Login or create account : User can login into already registered account or create a new account to login. If password is correct and verified then user can login into account immediately. If password is wrong then it will show wrong password message and it will go back to login page.',
 '',
 '',
 '2) Get bid history : User can see and get his/her bidding history.',
 '3) Get list of auction : User can see item list which is to be auctioned. ',
 '4) Place an item for bid : User, who is an owner, can place an item for bidding in auction for   other users-Date : When the user place the item for bidding, that date will be saved in database . Time : When the user place the item for bidding, that time will be saved in database. Picture of item : The user have to give pictures of the item that he/she want to place for bid. Withdraw: The user can withdraw the item that has gone for bidding from auction. Initial Price: The user can set t

In [86]:
func_req_list

['\ufeff1. SET UP ACCOUNT : When the user wants to enter into this software he/she should go through the account setup phrase. In this section the user will provide his/her personal details like home address, email, phone number, DOB and set up a password for the account,The password criteria should match as follows:  It should contain an uppercase alphabet, at least one special character, a numeric value.  It should be at least 8 character long.',
 '2. SIGN IN : Here the user can sign in to previously registered account by providing email address and password.  If login credentials are valid the user is directed to the app home page.  If there the credentials are invalid an error login message is displayed.  The user can reset his/her password by providing the registered email address. A password reset link is send to the user via email.  For sign in internet availability is required',
 '3. GET BID HISTORY : All user can get recent 100 bid history (picture of an item, initial price, a

## **Data Preprocessing**

In [87]:

def prep(list1):
  for i in range(len(list1)):
    list1[i] = list1[i].lower()
    list1[i] = ''.join((x for x in list1[i] if not x.isdigit()))
    list1[i] = re.sub(r"[^a-zA-Z0-9 ]", "", list1[i])
    list1[i] = re.sub(r'^https?:\/\/.*[\r\n]*', '', list1[i], flags=re.MULTILINE)
    list1[i] = re.sub(r'[^\w\s]', '', list1[i])
    text_tokens = word_tokenize(list1[i])
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    list1[i] = (" ").join(tokens_without_sw)
    res = list(filter(None, list1))
    x=set(res)

  return x

In [88]:
prep_use_case=prep(use_case_list)
prep_func_req=prep(func_req_list)

In [89]:
prep_use_case

{'cancel auction admin cancel whole auction anytime',
 'customer support user faces problem auction heshe go customer support help complain',
 'get bid history user see get hisher bidding history',
 'get list auction user see item list auctioned',
 'item approval admin approves item comes bidding monitor auction admin must monitor auction security authorization whole auction process authorized department keeps look auction security purpose helps user problems',
 'join auction user join auction successfully login bid item bid item successfully joining auction user bid item winner user bids item successfully bidding user becomes winner gets hand item winning heshe clear payment gets payment receipt gives feedback auction',
 'list item admin keeps list items presenting auction bidding',
 'list user admin keeps list users participating auction',
 'login admin enter auction logging',
 'login create account user login already registered account create new account login password correct verif

In [90]:
len(prep_use_case)

15

In [91]:
prep_func_req

{'customer support users problem auction heshe help customer support heshe write problem complain customer support admin security team try best help user complain browser related handled admin team first verify complain complain valid action complain payment relatedfraud case handled security team',
 'get bid history user get recent bid history picture item initial price auction pricedate time auctionuser anonymous old history automatically get deleted new bid added',
 'get list auction users get upcoming auction list product date time',
 'join auction user join auction heshe join minutes auction started heshe bid item bidding heshe winner heshe facility pay via hisher debit credit card net banking within minutes opportunity give feedback whole process heshe give hisher permanent address sending product payment heshe receive auction receipt payment receipt',
 'monitor auction user view product details features bidding time view biddings without participating',
 'place item bid user pla

In [92]:
len(prep_func_req)

8

## **For Weight Function**

In [93]:
def action_determining(text):
  nlp = spacy.load("en_core_web_sm")
  j=0
  id=[]
  lst=[]
  for i in text:
  
    doc1 = nlp(i)
    x1=[]
  
    id.append(j)
    for chunk in doc1.noun_chunks:
      if chunk.root.dep_ == 'dobj' or chunk.root.dep_=='comp'or chunk.root.dep_=='advmod' or chunk.root.dep_=='amod':
        y=chunk.text + ' ' + chunk.root.head.text

        x1.append(y)
    x2=set(x1)
    x3=list(x2)
    lst.append(x3)
    j=j+1
  action_per_sen = pd.DataFrame({'Index' : id,
                            'Actions' : lst})
  action_per_sen=action_per_sen[~action_per_sen.Actions.str.len().eq(0)]
  return action_per_sen

In [94]:
action_per_sen_use=action_determining(prep_use_case)
action_per_sen_use

,Index,Actions
0,0,"[user auction admin remove, remove remove, use..."
1,1,[whole auction cancel]
2,2,[join bidding need]
3,3,[auction user bid item winner user bids item j...
5,5,"[login page go, verified user login account co..."
6,6,[history bidding]
7,7,"[details items keeps, item sending, informatio..."
8,8,[user problems helps]
9,9,"[auction bidding presenting, list items keeps]"
11,11,"[auction receipt auctioning item gets, initial..."


In [95]:
action_per_sen_func=action_determining(prep_func_req)
action_per_sen_func

,Index,Actions
0,0,"[previously registered account sign, user send..."
1,1,"[uppercase alphabet contain, hisher personal d..."
2,2,[complain complain valid action complain payme...
3,3,[recent bid history picture item initial price...
4,4,"[auction receipt payment receipt receive, perm..."
5,5,[bidding time view biddings features]
7,7,"[particular date time give, product bidding we..."


In [96]:
from keybert import KeyBERT
kw_model = KeyBERT(model='all-mpnet-base-v2')
def keyword_determining(text):
  id=[]
  kw=[]
  text=list(text)
  for i in range(len(text)):
    keywords = kw_model.extract_keywords(text[i], 

                                      keyphrase_ngram_range=(2, 2), 

                                      stop_words='english', 

                                      highlight=False,

                                      top_n=5)
    id.append(i)
    keywords_list= list(dict(keywords).keys())
    kw.append(keywords_list)
    keyword_per_sen = pd.DataFrame({'index' : id,
                            'Keywords' : kw})
  return keyword_per_sen

In [97]:
keyword_per_sen_use=keyword_determining(prep_use_case)
keyword_per_sen_use

,index,Keywords
0,0,"[user auction, auction admin, users auction, r..."
1,1,"[cancel auction, auction admin, admin cancel, ..."
2,2,"[monitor auction, auction user, join bidding, ..."
3,3,"[auction user, joining auction, bidding user, ..."
4,4,"[heshe customer, auction heshe, problem auctio..."
5,5,"[password wrong, wrong password, password corr..."
6,6,"[bid history, bidding history, history user, h..."
7,7,"[item buyer, item sending, details item, sendi..."
8,8,"[auction admin, item approval, authorization a..."
9,9,"[auction bidding, presenting auction, item adm..."


In [98]:
keyword_per_sen_func=keyword_determining(prep_func_req)
keyword_per_sen_func

,index,Keywords
0,0,"[sign user, credentials invalid, registered ac..."
1,1,"[password criteria, accountthe password, set p..."
2,2,"[complain customer, customer support, user com..."
3,3,"[bid history, auctionuser anonymous, price auc..."
4,4,"[join auction, receive auction, auction starte..."
5,5,"[view biddings, monitor auction, biddings part..."
6,6,"[list auction, auction list, upcoming auction,..."
7,7,"[product bidding, online bidding, bidding webs..."


In [99]:
keyword_per_sen_use['Keywords']

0     [user auction, auction admin, users auction, r...
1     [cancel auction, auction admin, admin cancel, ...
2     [monitor auction, auction user, join bidding, ...
3     [auction user, joining auction, bidding user, ...
4     [heshe customer, auction heshe, problem auctio...
5     [password wrong, wrong password, password corr...
6     [bid history, bidding history, history user, h...
7     [item buyer, item sending, details item, sendi...
8     [auction admin, item approval, authorization a...
9     [auction bidding, presenting auction, item adm...
10                                                   []
11    [bidding auction, price auctioning, bid user, ...
12    [auction user, list auction, list auctioned, u...
13    [auction logging, login admin, enter auction, ...
14    [participating auction, list user, admin keeps...
Name: Keywords, dtype: object

In [100]:
keyword_per_sen_func['Keywords']

0    [sign user, credentials invalid, registered ac...
1    [password criteria, accountthe password, set p...
2    [complain customer, customer support, user com...
3    [bid history, auctionuser anonymous, price auc...
4    [join auction, receive auction, auction starte...
5    [view biddings, monitor auction, biddings part...
6    [list auction, auction list, upcoming auction,...
7    [product bidding, online bidding, bidding webs...
Name: Keywords, dtype: object

In [101]:
len(keyword_per_sen_use)

15

In [ ]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
use_case_no=[]
func_rq_no=[]
kw=[]
kw0=[]
kw1=[]
kw2=[]
kw3=[]
kw4=[]
avg=[]
c=0
for i in range(len(keyword_per_sen_use)):
  p=len(keyword_per_sen_use['Keywords'][i])

  for j in range(p):
    text1=keyword_per_sen_use['Keywords'][i][j]
    text_embeddings=model.encode(text1)

    for k in range(len(keyword_per_sen_func)):
      use_case_no.append(i)
      kw.append(j)
      func_rq_no.append(k)
      x=keyword_per_sen_func['Keywords'][k][0]
      text_embeddings1=model.encode(x)
      kw0.append(cosine_similarity(text_embeddings.reshape(1, -1),text_embeddings1.reshape(1, -1)))
      x=keyword_per_sen_func['Keywords'][k][1]
      text_embeddings1=model.encode(x)
      kw1.append(cosine_similarity(text_embeddings.reshape(1, -1),text_embeddings1.reshape(1, -1)))
      x=keyword_per_sen_func['Keywords'][k][2]
      text_embeddings1=model.encode(x)
      kw2.append(cosine_similarity(text_embeddings.reshape(1, -1),text_embeddings1.reshape(1, -1)))
      x=keyword_per_sen_func['Keywords'][k][3]
      text_embeddings1=model.encode(x)
      kw3.append(cosine_similarity(text_embeddings.reshape(1, -1),text_embeddings1.reshape(1, -1)))
      x=keyword_per_sen_func['Keywords'][k][4]
      text_embeddings1=model.encode(x)
      kw4.append(cosine_similarity(text_embeddings.reshape(1, -1),text_embeddings1.reshape(1, -1)))
      add=kw0[c]+kw1[c]+kw2[c]+kw3[c]+kw4[c]
      avg.append(add)
      c=c+1
  score_df = pd.DataFrame({'Use_case_sen_no' : use_case_no,
                          'Usecase_Keyword_no' : kw,
                          'Func_req_sen_no' : func_rq_no,
                          'FKeyword0' : kw0,
                          'FKeyword1' : kw1,
                          'FKeyword2' : kw2,
                          'FKeyword3' : kw3,
                          'FKeyword4' : kw4,
                          'Sum' : avg})

In [103]:
score_df

,Use_case_sen_no,Usecase_Keyword_no,Func_req_sen_no,FKeyword0,FKeyword1,FKeyword2,FKeyword3,FKeyword4,Sum
0,0,0,0,[[0.5538246]],[[0.37079048]],[[0.70943594]],[[0.6088769]],[[0.6470491]],[[2.8899772]]
1,0,0,1,[[0.5112468]],[[0.57938516]],[[0.52958405]],[[0.59063447]],[[0.7401703]],[[2.9510207]]
2,0,0,2,[[0.57016236]],[[0.60215986]],[[0.6060198]],[[0.5401988]],[[0.5049378]],[[2.8234787]]
3,0,0,3,[[0.62423384]],[[0.7139298]],[[0.87501967]],[[0.7711016]],[[0.6952592]],[[3.679544]]
4,0,0,4,[[0.832621]],[[0.84943515]],[[0.7543427]],[[0.95838255]],[[0.7478845]],[[4.142666]]
...,...,...,...,...,...,...,...,...,...
539,14,4,3,[[0.5883615]],[[0.46062124]],[[0.56963825]],[[0.5244074]],[[0.43075633]],[[2.5737846]]
540,14,4,4,[[0.66731596]],[[0.6290156]],[[0.55318195]],[[0.67248154]],[[0.74110055]],[[3.2630956]]
541,14,4,5,[[0.7503946]],[[0.5509318]],[[0.88183415]],[[0.7693007]],[[0.67248154]],[[3.6249428]]
542,14,4,6,[[0.53729177]],[[0.5695889]],[[0.48267275]],[[0.69157064]],[[0.57514745]],[[2.8562715]]


In [104]:
index=[]
score=[]
for i in range(len(prep_use_case)):
  for j in range(len(prep_func_req)):
    s=0
    for k in range(len(score_df)):
      if score_df['Use_case_sen_no'][k]==i and score_df['Func_req_sen_no'][k]==j:
        s=s+score_df['Sum'][k]
    f=i*10+j
    index.append(f)
    score.append(s)
    weight_list = pd.DataFrame({ 'Index' : index,
                                'Score' : score})

In [105]:
weight_list

,Index,Score
0,0,[[14.272407]]
1,1,[[12.554375]]
2,2,[[14.580261]]
3,3,[[16.211784]]
4,4,[[17.63556]]
...,...,...
115,143,[[15.206822]]
116,144,[[17.268444]]
117,145,[[18.352474]]
118,146,[[15.717223]]


In [106]:
prep_use_case=list(prep_use_case)
prep_func_req=list(prep_func_req)
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
uc=[]
fr=[]
score=[]
for i in range(len(prep_use_case)):
  for j in range(len(prep_func_req)):

    p=prep_use_case[i]
    text_embeddings1=model.encode(p)
    q=prep_func_req[j]
    text_embeddings2=model.encode(q)
    similarities=cosine_similarity(text_embeddings1.reshape(1, -1),text_embeddings2.reshape(1, -1))
    uc.append(i)
    fr.append(j)
    score.append(similarities)
per_sen_score = pd.DataFrame({'Use_case_sen_no' : uc,
                          'Func_req_sen_no' : fr,
                          'score' : score})

In [107]:
per_sen_score

,Use_case_sen_no,Func_req_sen_no,score
0,0,0,[[0.37485823]]
1,0,1,[[0.40062433]]
2,0,2,[[0.5059418]]
3,0,3,[[0.65811205]]
4,0,4,[[0.5030765]]
...,...,...,...
115,14,3,[[0.56832284]]
116,14,4,[[0.69251287]]
117,14,5,[[0.7326848]]
118,14,6,[[0.7274406]]


In [108]:
uc=[]
score=[]
k=0
for i in range(len(prep_use_case)):
  s=[]
  for j in range(len(prep_func_req)):
    c=per_sen_score['score'][k] * weight_list['Score'][k]
    s.append(c)
    k=k+1
  uc.append(i)
  score.append(s)
  weight_df = pd.DataFrame({'Use_case_sen_no' : uc,
                          'score' : score})

In [109]:
weight_df

,Use_case_sen_no,score
0,0,"[[[5.350129]], [[5.029588]], [[7.376764]], [[1..."
1,1,"[[[3.3194623]], [[3.003991]], [[2.624991]], [[..."
2,2,"[[[5.6689587]], [[7.276594]], [[6.538337]], [[..."
3,3,"[[[8.269174]], [[7.531787]], [[10.156097]], [[..."
4,4,"[[[6.374519]], [[4.913282]], [[11.972927]], [[..."
5,5,"[[[11.601397]], [[12.401875]], [[6.415479]], [..."
6,6,"[[[6.6923676]], [[7.1956153]], [[9.18822]], [[..."
7,7,"[[[8.402185]], [[7.596005]], [[12.364016]], [[..."
8,8,"[[[8.149111]], [[6.6551924]], [[11.135922]], [..."
9,9,"[[[6.0770807]], [[6.247594]], [[10.002176]], [..."


In [110]:
c=per_sen_score['score'][0] * weight_list['Score'][0]

In [111]:
c

array([[5.350129]], dtype=float32)

In [112]:
prep_use_case[1]+' '+prep_func_req[7]

'cancel auction admin cancel whole auction anytime place item bid user place item online bidding particular date time item placed product bidding website within heshe give particular date time bidding picture item heshe set initial price item heshe right withdraw product'

In [113]:
keyword_per_sen_use['Keywords'][1]+keyword_per_sen_func['Keywords'][7]

['cancel auction',
 'auction admin',
 'admin cancel',
 'auction anytime',
 'product bidding',
 'online bidding',
 'bidding website',
 'time bidding',
 'item bid']

In [114]:
keyword_per_sen_func['Keywords'][7]

['product bidding',
 'online bidding',
 'bidding website',
 'time bidding',
 'item bid']

In [115]:
uc=[]
sen=[]
key=[]
for i in range(len(weight_df)):
  s=[]
  p=[]
  for j in range(len(weight_df['score'][i])):
    if weight_df['score'][i][j]>11:
      x=prep_use_case[i]+' '+prep_func_req[j]
      y=keyword_per_sen_use['Keywords'][i]+keyword_per_sen_func['Keywords'][j]
      s.append(x)
      p.append(y)
  uc.append(i)
  sen.append(s)
  key.append(p)
  sen_output = pd.DataFrame({'Use_case_sen_no' : uc,
                          'Sentences' : sen,
                          'Keywords' : key})

In [116]:
sen_output

,Use_case_sen_no,Sentences,Keywords
0,0,[remove user auction admin remove user users a...,"[[user auction, auction admin, users auction, ..."
1,1,[],[]
2,2,[monitor auction user monitor auction though h...,"[[monitor auction, auction user, join bidding,..."
3,3,[join auction user join auction successfully l...,"[[auction user, joining auction, bidding user,..."
4,4,[customer support user faces problem auction h...,"[[heshe customer, auction heshe, problem aucti..."
5,5,[login create account user login already regis...,"[[password wrong, wrong password, password cor..."
6,6,[get bid history user see get hisher bidding h...,"[[bid history, bidding history, history user, ..."
7,7,[note details item buyer delivery admin keeps ...,"[[item buyer, item sending, details item, send..."
8,8,[item approval admin approves item comes biddi...,"[[auction admin, item approval, authorization ..."
9,9,[list item admin keeps list items presenting a...,"[[auction bidding, presenting auction, item ad..."


## FOR FEEDBACK FUNCTION

In [117]:
def dependency_parsing(text1,text2):
  nlp = spacy.load("en_core_web_sm")
  doc1 = nlp(text1)
  x1=[]
  for chunk in doc1.noun_chunks:
    if chunk.root.dep_ == 'dobj' or chunk.root.dep_=='comp'or chunk.root.dep_=='advmod' or chunk.root.dep_=='amod':
      y=chunk.text + ' ' + chunk.root.head.text
      x1.append(y)
  doc2 = nlp(text2)
  x2=[]
  for chunk in doc2.noun_chunks:
    if chunk.root.dep_ == 'dobj'or chunk.root.dep_=='comp'or chunk.root.dep_=='advmod' or chunk.root.dep_=='amod':
      y=chunk.text + ' ' + chunk.root.head.text
      x2.append(y)
  for i in range(len(x1)):
    x1[i] = x1[i].lower()
    x1[i] = ''.join((x for x in x1[i] if not x.isdigit()))
    x1[i] = re.sub(r"[^a-zA-Z0-9 ]", "", x1[i])
    x1[i] = re.sub(r'^https?:\/\/.*[\r\n]*', '', x1[i], flags=re.MULTILINE)
    x1[i] = re.sub(r'[^\w\s]', '', x1[i])
    text_tokens = word_tokenize(x1[i])
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    x1[i] = (" ").join(tokens_without_sw)
  for i in range(len(x2)):
    x2[i] = x2[i].lower()
    x2[i] = ''.join((x for x in x2[i] if not x.isdigit()))
    x2[i] = re.sub(r"[^a-zA-Z0-9 ]", "", x2[i])
    x2[i] = re.sub(r'^https?:\/\/.*[\r\n]*', '', x2[i], flags=re.MULTILINE)
    x2[i] = re.sub(r'[^\w\s]', '', x2[i])
    text_tokens = word_tokenize(x2[i])
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    x2[i] = (" ").join(tokens_without_sw)
  x1=list(set(x1))
  x2=list(set(x2))
  return x1,x2

In [118]:
f1,f2=dependency_parsing(use_case,func_req)

In [119]:
f1

['user bidding',
 'details keeps',
 'new account create',
 'winning',
 'auction receipt gets',
 'auction enter',
 'auction monitor',
 'history bid',
 'payment receipt gets',
 'item bids',
 'item list see',
 'whole auction cancel',
 'wrong password message show',
 'payment clear',
 'list get',
 'item approves',
 'bidding join',
 'auction join',
 'item place',
 'item auctioning',
 'item bid',
 'pictures give',
 'account create',
 'initial price set',
 'item sending',
 'details note',
 'item bade',
 'hand gets',
 'problem faces',
 'list keeps',
 'user remove',
 'look keeps',
 'feedback gives',
 'auction joining',
 'hisher bidding history get',
 'user helps',
 'final price gets',
 'item withdraw']

In [120]:
def keyword_extraction(text1,text2):
  text1 = ' '.join(text1)
  text2 = ' '.join(text2)
  n_gram_range = (3, 3)
  stop_words = "english"
  top_n = 5
  model = SentenceTransformer('distilbert-base-nli-mean-tokens')

  # Extract candidate words/phrases
  count1 = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([text1])
  candidates1 = count1.get_feature_names()
  doc_embedding1 = model.encode([text1])
  candidate_embeddings1 = model.encode(candidates1)
  distances1 = cosine_similarity(doc_embedding1, candidate_embeddings1)
  keywords1 = [candidates1[index1] for index1 in distances1.argsort()[0][-top_n:]]

  count2 = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([text2])
  candidates2 = count2.get_feature_names()
  doc_embedding2 = model.encode([text2])
  candidate_embeddings2 = model.encode(candidates2)
  distances2 = cosine_similarity(doc_embedding2, candidate_embeddings2)
  keywords2 = [candidates2[index2] for index2 in distances2.argsort()[0][-top_n:]]
  return keywords1,keywords2

In [121]:
def feedback(text1,text2):
  # depen_par fn + keyword_ext + similarity
  x1, x2 = dependency_parsing(text1,text2)
  k1, k2 = keyword_extraction(x1,x2)

  print("Functional Requirements:",k1)
  print("Modified Text:",k2)


In [122]:
feedback(func_req,sugg)

Functional Requirements: ['date hisher password', 'auction list help', 'password reset registered', 'hisher password reset', 'reset registered email']
Modified Text: ['recent bid history', 'list recent bid', 'auction list recent', 'upcoming auction list']
